![imagen](./img/python.jpg)

# SQL en Python
Podrás atacar a una base de datos SQL desde muchas plataformas/lenguajes. Por supuesto, Python es uno de ellos. Para ello **utilizaremos el módulo `sqlite3`**, que viene incluido en la suite de paquetes de Anaconda.


1. [Configuración del entorno](#1.-Configuración-del-entorno)
2. [Modelo de datos](#2.-Modelo-de-datos)
3. [Queries](#3.-Queries)

    3.1 [Primera query](#3.1-Primera-query)
    
    3.2 [SELECT](#3.2-SELECT)
    
    3.3 [LIMIT](#3.3-LIMIT)
    
    3.4 [DISTINCT](#3.4-DISTINCT)
    
    3.5 [WHERE](#3.5-WHERE)
    
    3.6 [ORDER BY](#3.6-ORDER-BY)
    
    3.7 [Agregaciones](#3.7-Agregaciones)
    
    3.8 [GROUP BY](#3.8-GROUP-BY)
    
    3.9 [JOIN](#3.9-JOIN)
    
    3.10 [VIEW y DROP](#3.10-VIEW-y-DROP)
    
4. [Errores](#4.-Errores)

5. [Resumen](#5.-Resumen)


## 1. Configuración del entorno
En este taller usaremos también *Pandas*, que es el módulo para tratamiento de datos tabular que más se usa en Python.

Importamos ambos paquetes

In [ ]:
# %pip install sqlite3 **NOT NEEDED: sqlite3 is included with Python standard library**

In [ ]:
# %pip install pysqlite3

In [1]:
import pandas as pd
import sqlite3

Lo primero que hacemos es **establecer conexión con la base de datos**. En este taller simplemente leeremos la base de datos de un archivo, pero lo normal es que tengamos que configurar la conexión a una base de datos de la empresa. Para ello existen otras librerías como `pyodbc` en el caso de un SQLServer o `cx_Oracle`, si el servidor es Oracle.

**¡Asegurate que tienes el archivo chinook.db en la ruta donde se encuentra este Notebook!**

In [2]:
import os
os.getcwd()

'/home/david/The_Bridge/2025-10-DS-FT-BIL/2025-OCT-BILBAO-FT-Data-Science/2-Data_Analysis/8-BBDD/SQL/Teoria/SQL_Python'

In [3]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("data/chinook.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

In [4]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

## 2. Modelo de datos
Antes de empezar a atacar una base de datos, tendremos que saber qué hay dentro, y para ello lo mejor es ver cómo es su **modelo de datos**

![imagen](./img/chinook_data_model.png)

Puedes ver todas las tablas que hay en la base de datos con la siguiente sentencia

In [7]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists
playlist_track
tracks
sqlite_stat1
films


In [8]:
sql_query("SELECT name FROM sqlite_master WHERE type='table'")

,name
0,albums
1,sqlite_sequence
2,artists
3,customers
4,employees
5,genres
6,invoices
7,invoice_items
8,media_types
9,playlists


## 3. Queries
En las *queries* o consultas, escribiremos nuestra sentencia SQL para poder traernos datos de la base de datos. Las queries tienen esta pinta:

```SQL
SELECT campo1, campo2, campo3...
FROM tabla
WHERE condiciones
```

Por supuesto, hay más sentencias propias de SQL que iremos viendo a lo largo del Notebook. Fíjate que las **palabras reservadas en SQL se suelen poner en mayúsculas**, para diferenciarlas del resto. No da error si se pone de otra manera, ya que SQL **no es *case sensitive***, pero sí se suele hacer así.

### 3.1 Primera query

In [9]:
query = '''
SELECT * 
FROM tracks
'''

df = sql_query(query)
df

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


In [10]:
query = open("queries/query_1.sql", "r").read()
print(query)
df = sql_query(query)
df

SELECT * 
FROM tracks;


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


In [11]:
pd.read_sql_query("SELECT * FROM tracks;",connection)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


1. Crea una carpeta llamada 'queries'
2. Crea 3 queries de sql, almacenadas en la carpeta 'queries'
3. Ejecuta las consultas de esas 3 queries y almacena los resultados en carpeta 'data'

In [12]:
os.getcwd()

'/home/david/The_Bridge/2025-10-DS-FT-BIL/2025-OCT-BILBAO-FT-Data-Science/2-Data_Analysis/8-BBDD/SQL/Teoria/SQL_Python'

In [13]:
os.listdir(os.getcwd() + "/queries")

['query_1.sql',
 'northwind_join_groupby_order.solved.sql',
 'chinook.solved.sql',
 'query_base.sql',
 'query_3.solved.sql',
 'subquery_between.sql',
 'query_1.solved.sql',
 'query_2.solved.sql']

### 3.2 SELECT
**La sentencia `SELECT` se usa para filtrar columnas**. Podremos incluso renombrar algunas columnas dentro del propio `SELECT`. Dos opciones:
* **SELECT * :** de esta forma lo que haremos es traer todos los campos de la tabla. NO es lo recomendable ya que hay tablas que pueden tener cientos de columnas y probablemente no necesitemos todas, por lo que afectará al rendimiento de la maquina y en definitiva al resto de usuarios. **Recuerda que las bases de datos son normalmente entornos compartidos por varios usuarios :)**
* **SELECT campo1, campo2...:** sentencia para escoger los campos concretos que necesitamos.

Además en la propia query podrás renombrar campos mediante la sintaxis `campo as nuevo_nombre`. **Si quieres poner espacios en el nombre del campo, tendrás que rodear el string con comillas dobles**

SQL no es sensible a mayusculas y minusculas.

In [14]:
query = '''
SELECT Name as "Nombre Cancion", composer as "Compositor"
FROM tracks
'''

sql_query(query)

,Nombre Cancion,Compositor
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson"
1,Balls to the Wall,None
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
3,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
4,Princess of the Dawn,Deaffy & R.A. Smith-Diesel
...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,None
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Franz Schubert
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",Claudio Monteverdi
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Wolfgang Amadeus Mozart


### 3.3 LIMIT
Se usa para acotar el número de registros de la query. Va siempre al final. Por ejemplo `LIMIT 10`

In [15]:
query = '''
SELECT Name as "Nombre Cancion", composer
FROM tracks
LIMIT 10
'''

sql_query(query)

,Nombre Cancion,Composer
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson"
1,Balls to the Wall,None
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
3,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
4,Princess of the Dawn,Deaffy & R.A. Smith-Diesel
5,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson"
6,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson"
7,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson"
8,Snowballed,"Angus Young, Malcolm Young, Brian Johnson"
9,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson"


### 3.4 DISTINCT
Se usa para obtener todos los registros únicos, es decir, sin duplicados. Muy útil tanto para eliminar dupicados, como para ver todas las casuísticas de un campo en concreto.

**Mucho cuidado con esta sentencia ya que si la tabla tiene miles o millones de registros, puede ralentizar mucho la query.**

In [16]:
query = '''
SELECT DISTINCT Composer
FROM tracks
'''

sql_query(query)

,Composer
0,"Angus Young, Malcolm Young, Brian Johnson"
1,None
2,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
3,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
4,Deaffy & R.A. Smith-Diesel
...,...
848,Carl Nielsen
849,Niccolò Paganini
850,Pietro Antonio Locatelli
851,Claudio Monteverdi


### 3.5 WHERE
Se usa para filtrar filas. Ejemplos de uso:
* **Un valor numérico**
    * UnitPrice = 0.99
    * UnitPrice >= 0.99
    * UnitPrice < 0.99
* **Un valor string**: Name = 'Restless and Wild'
* **Varios valores**: GenreId in (1, 5, 12)
* **string contenido**:
    * strings que empiecen por 'A': Name like 'A%'
    * strings que acaben en 'A': Name like '%A'
    * strings que lleven 'A' en algun punto: Name like '%A%'
* **Distinto de**: UnitPrice <> 0.99

Probamos el `WHERE`

In [17]:
query = '''
SELECT * 
FROM tracks
WHERE unitprice > 0.99
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,2819,Battlestar Galactica: The Story So Far,226,3,18,None,2622250,490750393,1.99
1,2820,Occupation / Precipice,227,3,19,None,5286953,1054423946,1.99
2,2821,"Exodus, Pt. 1",227,3,19,None,2621708,475079441,1.99
3,2822,"Exodus, Pt. 2",227,3,19,None,2618000,466820021,1.99
4,2823,Collaborators,227,3,19,None,2626626,483484911,1.99
...,...,...,...,...,...,...,...,...,...
208,3362,"There's No Place Like Home, Pt. 1",261,3,21,None,2609526,522919189,1.99
209,3363,"There's No Place Like Home, Pt. 2",261,3,21,None,2497956,523748920,1.99
210,3364,"There's No Place Like Home, Pt. 3",261,3,21,None,2582957,486161766,1.99
211,3428,Branch Closing,251,3,22,None,1814855,360331351,1.99


Veamos qué tal funciona el `LIKE`

In [18]:
query = '''
SELECT * 
FROM tracks
WHERE composer LIKE '%Brian Johnson%'
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99
5,10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99
6,11,C.O.D.,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",199836,6566314,0.99
7,12,Breaking The Rules,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263288,8596840,0.99
8,13,Night Of The Long Knives,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205688,6706347,0.99
9,14,Spellbound,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",270863,8817038,0.99


In [19]:
query = '''
SELECT DISTINCT composer 
FROM tracks
WHERE composer LIKE '%Brian Johnson%'
'''

sql_query(query)

,Composer
0,"Angus Young, Malcolm Young, Brian Johnson"


Compliquemos un poco más el `WHERE`

In [20]:
query = '''
SELECT * 
FROM tracks
WHERE (unitprice > 0.99 or bytes > 100000000) and genreid in (21, 22, 23) and Name LIKE '%al%'
'''

df = sql_query(query)
df

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,2844,Better Halves,228,3,21,None,2573031,549353481,1.99
1,2849,Fallout,228,3,21,None,2594761,501145440,1.99
2,2862,The Glass Ballerina,229,3,21,None,2637458,535729216,1.99
3,2882,Lost Survival Guide,229,3,21,None,2632590,486675063,1.99
4,2899,The Man from Tallahassee,229,3,21,None,2637637,550893556,1.99
5,2922,"Live Together, Die Alone, Pt. 1",231,3,21,None,2478041,457364940,1.99
6,3211,Traveling Salesmen,251,3,22,None,1289039,250822697,1.99
7,3219,Product Recall,251,3,22,None,1268268,251208610,1.99


In [21]:
query = '''
SELECT * 
FROM tracks
WHERE Name LIKE 'You%'
'''

df = sql_query(query)
df

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,39,You Oughta Know,6,1,1,Alanis Morissette & Glenn Ballard,249234,8196916,0.99
1,44,You Learn,6,1,1,Alanis Morissette & Glenn Ballard,239699,7824837,0.99
2,50,You Oughta Know (Alternate),6,1,1,Alanis Morissette & Glenn Ballard,491885,16008629,0.99
3,99,Your Time Has Come,11,1,4,"Cornell, Commerford, Morello, Wilk",255529,8273592,0.99
4,337,You Shook Me,30,1,1,J B Lenoir/Willie Dixon,315951,10249958,0.99
5,349,You Shook Me(2),30,1,1,J B Lenoir/Willie Dixon,619467,20138673,0.99
6,549,You Fool No One,43,1,1,Coverdale/Lord/Paice,804101,26369966,0.99
7,623,You Fool No One (Alternate Version),50,1,1,Blackmore/Coverdale/Lord/Paice,763924,24887209,0.99
8,753,You Keep On Moving,58,1,1,Coverdale/Hughes,319111,10447868,0.99
9,812,You Can't Do it Right (With the One You Love),65,1,1,D.Coverdale/G.Hughes/Glenn Hughes/R.Blackmore/...,203755,6709579,0.99


### 3.6 ORDER BY
Podemos **ordenar la tabla por el campo/s que queramos**. Por defecto ordena alfabéticamente los strings y de menor a mayor los tipos numéricos. Si quieres que ordene al revés, tienes que poner `DESC` de la forma `ORDER BY campo DESC`

In [22]:
query = '''
SELECT * 
FROM tracks
ORDER BY name DESC;
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1077,Último Pau-De-Arara,85,1,10,Corumbá/José Gumarães/Venancio,200437,6638563,0.99
1,1073,Óia Eu Aqui De Novo,85,1,10,None,219454,7469735,0.99
2,2078,Óculos,169,1,7,None,219271,7262419,0.99
3,3496,"Étude 1, In C Major - Preludio (Presto) - Liszt",340,4,24,None,51780,2229617,0.99
4,333,É que Nessa Encarnação Eu Nasci Manga,29,1,9,Lucina/Luli,196519,6568081,0.99
...,...,...,...,...,...,...,...,...,...
3498,3254,#9 Dream,255,2,9,None,278312,4506425,0.99
3499,109,#1 Zero,11,1,4,"Cornell, Commerford, Morello, Wilk",299102,9731988,0.99
3500,3412,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",281,2,24,Wolfgang Amadeus Mozart,348971,5760129,0.99
3501,2918,"""?""",231,3,19,None,2782333,528227089,1.99


### 3.7 Agregaciones
En ocasiones nos interesa obtener algún estadístico como el máximo de un campo, su desviación estándar o simplemente un conteo de registros no nulos. Para ello podemos usar funciones como `MAX`, `COUNT` o `AVG`. En [esta página](https://www.sqlservertutorial.net/sql-server-aggregate-functions/) encontrarás un resumen con las principales funciones.

In [23]:
# Aggregations
query = '''
SELECT COUNT(*)
FROM tracks
WHERE name LIKE 'You%';
'''

sql_query(query)

,COUNT(*)
0,38


In [24]:
query = '''
SELECT AVG(unitprice)
FROM invoice_items;
'''

sql_query(query)

,AVG(unitprice)
0,1.039554


### 3.8 GROUP BY
Es una sentencia muy útil para **calcular agregados en función de otro campo**. Por ejemplo, para calcular el total del precio unitario en función del género


In [25]:
query = '''
SELECT GenreId, SUM(unitprice) as TOT_PRICE
FROM tracks
GROUP BY GenreId
ORDER BY TOT_PRICE DESC
LIMIT 10;
'''

sql_query(query)

,GenreId,TOT_PRICE
0,1,1284.03
1,7,573.21
2,3,370.26
3,4,328.68
4,19,185.07
5,2,128.70
6,21,127.36
7,6,80.19
8,24,73.26
9,14,60.39


In [28]:
query = '''
SELECT g.Name, SUM(t.unitprice) as TOT_PRICE
FROM genres as g
LEFT JOIN tracks as t ON g.GenreId = t.GenreId
GROUP BY g.Name
ORDER BY TOT_PRICE DESC
--LIMIT 10;
'''

sql_query(query)

,Name,TOT_PRICE
0,Rock,1284.03
1,Latin,573.21
2,Metal,370.26
3,Alternative & Punk,328.68
4,TV Shows,185.07
5,Jazz,128.70
6,Drama,127.36
7,Blues,80.19
8,Classical,73.26
9,R&B/Soul,60.39


O calcular cuantas canciones hay por compositor

In [29]:
query = '''
SELECT composer, COUNT(trackid)
FROM tracks
WHERE Composer IS NOT NULL
GROUP BY composer
ORDER BY 2 DESC
LIMIT 100
;
'''

sql_query(query)

,Composer,COUNT(trackid)
0,Steve Harris,80
1,U2,44
2,Jagger/Richards,35
3,Billy Corgan,31
4,Kurt Cobain,26
...,...,...
95,"James Hetfield, Lars Ulrich, Kirk Hammett",7
96,Ian Gillan/Ian Paice/Jon Lord/Ritchie Blckmore...,7
97,Harris,7
98,Green Day,7


### 3.9 JOIN
Hasta ahora hemos hecho queries sobre una única tabla, pero **¿y si queremos juntar datos de varias tablas?** Para eso están los `JOIN`s. Para ello **necesitas tener uno o varios campos comunes entre ambas tablas, que denominan CLAVES**.

**¿Cuándo usarlos?** Por ejemplo, si tenemos una tabla con un conjunto de clientes y necesitamos añadirles campos nuevos, tendremos que acudir a otras tablas donde esté ese identificador de cliente y aplicar un `JOIN`. Es lo que se conoce como *pegar campos* a otra tabla.

O imagina que tienes una tabla con todos tus pedidos, con muchos campos(ciudad, dirección, cliente...) y en otra tabla únicamente los números de pedido que no se llegaron a entregar. Si quieres filtrar dentro de tu tabla total de pedidos los que no se llegaron a entregar, podrías aplicar un `INNER JOIN` de manera que te quedes con lo común en ambas tablas, siendo tu clave el identificativo del pedido..

Existen varios tipos de JOINs:

![imagen](./img/joins.jpg)

![imagen](./img/chinook_data_model.png)

In [30]:
#INNER JOIN
query = '''
SELECT a.name as track_name, a.composer, b.name as genre_name
FROM tracks AS a
INNER JOIN genres AS b
ON a.genreid = b.genreid
;
'''

sql_query(query)

,track_name,Composer,genre_name
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",Rock
1,Balls to the Wall,None,Rock
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",Rock
3,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",Rock
4,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,Rock
...,...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,None,Classical
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Franz Schubert,Classical
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",Claudio Monteverdi,Classical
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Wolfgang Amadeus Mozart,Classical


In [31]:
query = '''
SELECT b.name as genre_name, SUM(a.unitprice) as TOT_PRICE
FROM tracks as a
INNER JOIN genres as b
ON a.GenreId=b.GenreId
GROUP BY b.name
ORDER BY TOT_PRICE DESC
LIMIT 10;
'''

sql_query(query)

,genre_name,TOT_PRICE
0,Rock,1284.03
1,Latin,573.21
2,Metal,370.26
3,Alternative & Punk,328.68
4,TV Shows,185.07
5,Jazz,128.70
6,Drama,127.36
7,Blues,80.19
8,Classical,73.26
9,R&B/Soul,60.39


![imagen](./img/chinook_data_model.png)

In [32]:

query = '''
SELECT a.trackid, a.name, a.composer, b.invoiceid
FROM tracks AS a
INNER JOIN invoice_items AS b
ON a.trackid = b.trackid
;
'''

sql_query(query)

,TrackId,Name,Composer,InvoiceId
0,2,Balls to the Wall,None,1
1,4,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",1
2,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",2
3,8,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",2
4,10,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",2
...,...,...,...,...
2235,3136,Looking For Love,Sykes,411
2236,3145,Sweet Lady Luck,Vandenberg,411
2237,3154,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,"Arlindo Cruz/Franco/Marquinhos PQD/Negro, Jove...",411
2238,3163,Samba pras moças,Grazielle/Roque Ferreira,411


In [33]:
query = '''
SELECT a.trackid, a.name, a.composer, b.invoiceid
FROM tracks AS a
RIGHT JOIN invoice_items AS b
ON a.trackid = b.trackid
;
'''

sql_query(query)

,TrackId,Name,Composer,InvoiceId
0,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",108
1,2,Balls to the Wall,None,1
2,2,Balls to the Wall,None,214
3,3,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",319
4,4,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",1
...,...,...,...,...
2235,3494,"Symphony No. 2, Op. 16 - ""The Four Temperamen...",Carl Nielsen,319
2236,3496,"Étude 1, In C Major - Preludio (Presto) - Liszt",None,108
2237,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,None,214
2238,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Franz Schubert,108


In [34]:

query = '''
SELECT name as track_title, albumid
FROM tracks
;
'''

sql_query(query)

,track_title,AlbumId
0,For Those About To Rock (We Salute You),1
1,Balls to the Wall,2
2,Fast As a Shark,3
3,Restless and Wild,3
4,Princess of the Dawn,3
...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",345
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346


In [38]:
#LEFT JOIN
query = '''
SELECT a.name as track_title , b.title as album_title
FROM tracks AS a
LEFT JOIN albums AS b
ON a.albumid = b.albumid
'''

df_lj = sql_query(query)
# df_lj[df_lj['album_title'].isna()]
df_lj

,track_title,album_title
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Balls to the Wall,Balls to the Wall
2,Fast As a Shark,Restless and Wild
3,Restless and Wild,Restless and Wild
4,Princess of the Dawn,Restless and Wild
...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Respighi:Pines of Rome
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Schubert: The Late String Quartets & String Qu...
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",Monteverdi: L'Orfeo
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Mozart: Chamber Music


In [39]:
query = '''
SELECT * 
FROM invoice_items AS a
LEFT JOIN tracks AS b
ON a.trackid = b.trackid
LIMIT 5;
'''

sql_query(query)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,1,2,0.99,1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
1,2,1,4,0.99,1,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
2,3,2,6,0.99,1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
3,4,2,8,0.99,1,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
4,5,2,10,0.99,1,10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99


In [40]:
query = '''
SELECT c.Country, SUM(t.UnitPrice) as tot 
FROM customers as c
LEFT JOIN Invoices as i
ON c.CustomerId = i.CustomerId
LEFT JOIN Invoice_Items as ii
ON i.InvoiceId = ii.InvoiceId
LEFT JOIN tracks as t
ON ii.TrackId = t.TrackId
GROUP BY c.Country;
'''

sql_query(query)

,Country,tot
0,Argentina,37.62
1,Australia,37.62
2,Austria,42.62
3,Belgium,37.62
4,Brazil,190.10
5,Canada,303.96
6,Chile,46.62
7,Czech Republic,90.24
8,Denmark,37.62
9,Finland,41.62


<table align="left">
 <tr><td width="80"><img src="./img/error.png" style="width:auto;height:auto"></td>
     <td style="text-align:left">
         <h3>ERRORES Acceso a base de datos que no existe</h3>
         
 </td></tr>
</table>

¡Cuidado! No da error, sino que crea una base de datos vacía

In [41]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("chinook8.db")

crsr = connection.cursor()

In [42]:
query = '''
 CREATE TABLE Persons (
     PersonID int,
     LastName varchar(255),
     FirstName varchar(255),
     Address varchar(255),
     City varchar(255)
 );
 '''
crsr.execute(query)

In [ ]:
query = '''
 SELECT * FROM Persons
 '''
crsr.execute(query).fetchall()

In [43]:
query = '''
 INSERT INTO Persons (PersonID, LastName, FirstName, Address, City)
 VALUES (1, "N", "M", "Recoletos", "Madrid");
'''
crsr.execute(query)

In [44]:
query = '''
 SELECT * FROM Persons
 '''
crsr.execute(query).fetchall()

[(1, 'N', 'M', 'Recoletos', 'Madrid')]

In [45]:
connection.commit()
connection.close()

<table align="left">
 <tr><td width="80"><img src="./img/error.png" style="width:auto;height:auto"></td>
     <td style="text-align:left">
         <h3>ERRORES en la query</h3>
         
 </td></tr>
</table>

Los errores son todos del mismo tipo: `OperationalError`. Esto nos indica que lo da el propio SQL, no es un error del intérprete de Python como tal. Ahora bien, aunque lo de SQL, devuelve un descriptivo con el tipo de error, que está bastante bien para ver por donde van los tiros.

In [46]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("chinook8.db")

crsr = connection.cursor()

In [47]:
query = '''
SELECT last_name 
FROM Persons
'''

sql_query(query)

OperationalError: no such column: last_name

In [48]:
query = '''
SELECT * 
FROM tracks
'''

sql_query(query)

OperationalError: no such table: tracks

In [49]:
query = '''
SELECT * 
FROM tracksssss
'''

sql_query(query)

OperationalError: no such table: tracksssss

In [50]:
query = '''
SELECT campo1, campo2
FROM tracks
'''

sql_query(query)

OperationalError: no such table: tracks

## 5. Resumen
**SQL es el estándar que se utiliza en el acceso a las BBDD relacionales**, por lo que en prácticamente cualquier empresa te vas a encontrar una BD de este tipo. La sintaxis es:

```SQL
SELECT campo1, campo2, campo3...
FROM tabla
WHERE condicionales
```

Y podrás realizar las siguientes acciones:
1. **Filtrar columnas**: mediante `SELECT`
2. **Renombrar campos**: mediante `SELECT campo as nuevo_nombre`
3. **Eliminar duplicados**: mediante `DISTINCT`
4. **Limitar el número de registros**: `LIMIT`
5. **Filtrar filas**: `WHERE`
6. **Ordenar una tabla**: `ORDER BY campo1, campo2` y `DESC`
7. **Agregar información**: sacar un KPI como un máximo o un conteo: `MAX`,`COUNT`,`AVG`...
8. **Agregado a nivel grupo**: con `GROUP BY`. Se calculan las funciones de agregación para cada grupo de una columna.
9. **Juntar datos**: mediante `JOIN`. Hay de varios tipos: `LEFT`, `RIGHT`, `FULL`, `INNER` y `OUTER`.
10. **Tablas temporales**: con `VIEW`
11. **Eliminar tablas o vistas**: con `DROP`

Y si te manejas de maravilla con la librería pandas, **siempre puedes obtener tus datos mediante una query con sqlite3, y tratarlos posteriormente en Python, mediante las funciones de pandas.**